In [1]:
import numpy as np
import pandas as pd
from pandas.io import sql
from sqlalchemy import create_engine
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%load_ext dotenv
%dotenv ../.env

In [3]:
cnx = create_engine(os.getenv('EDMDB'))

Are there any invalid proxcode values on PLUTO? There are some NULL values.

In [4]:
proxcode_counts = pd.read_sql_query('''SELECT "ProxCode", COUNT(*)
FROM dcp.pluto202
GROUP BY "ProxCode";''', cnx)
proxcode_counts

,ProxCode,count
0,0,141749
1,1,313774
2,2,200383
3,3,200940
4,None,375


In [5]:
sql.execute('''DROP TABLE IF EXISTS dcp.buildings_23;
DROP TABLE IF EXISTS dcp.attached_no_neighbor;''', cnx)

Get buildings and geometries for lots marked as attached or semi-attached. Limit analysis to those lots having only one building.

In [6]:
sql.execute('''CREATE TABLE dcp.buildings_23 AS
    SELECT CAST(p."BBL" AS TEXT) AS bbl, bin, f.geom as building_geom, p.geom as lot_geom
    FROM dcp.bldg_footprints f, dcp.pluto202 p
    WHERE f.mpluto_bbl = CAST(p."BBL" AS TEXT)
    AND "ProxCode" IN ('2', '3')
    AND "NumBldgs" = 1;
CREATE INDEX lot_geom_idx
  ON dcp.buildings_23
  USING GIST (lot_geom);
CREATE INDEX building_geom_idx
  ON dcp.buildings_23
  USING GIST (building_geom);
''', cnx)

Get buildings whose ProxCode is either 'attached' or 'semi-attached', yet have no buildings abutting them.
Note that ST_Intersects seemed to work better here than ST_Touches; I'm not sure why.

In [ ]:
sql.execute('''CREATE TABLE dcp.attached_no_neighbor AS
SELECT b1.bbl, b1.bin, b1.building_geom, b1.lot_geom, COALESCE(x.count,0) AS count
FROM dcp.buildings_23 b1
LEFT JOIN (
	SELECT b1.bbl as bbl, count(*) AS count
	FROM dcp.bldg_footprints b2, dcp.buildings_23 b1
	WHERE st_intersects(b2.geom, b1.building_geom)
	AND b2.mpluto_bbl <> b1.bbl
	AND b2.bin <> b1.bin
	GROUP BY b1.bbl
) x
ON b1.bbl = x.bbl
WHERE COALESCE(x.count,0) = 0;''', cnx)

In [ ]:
no_abutting_buildings = pd.read_sql_query('''SELECT COUNT(*) FROM dcp.attached_no_neighbor;''', cnx)
no_abutting_buildings

Of these buildings who have no attached building, how many have a building within a foot?

In [ ]:
with_onefoot_neighbors = pd.read_sql_query('''SELECT a.*, ST_Distance(a.building_geom, b.geom) AS distance
FROM dcp.attached_no_neighbor a, dcp.bldg_footprints b
WHERE ST_DWithin(a.building_geom, b.geom, 1)
AND a.bin <> b.bin
AND ST_Distance(a.building_geom, b.geom) > 0
ORDER BY 6 desc;''', cnx)
with_onefoot_neighbors